In [107]:
# spell correction
# normalizer

In [108]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import re
from hazm import InformalNormalizer, Normalizer

In [109]:
tqdm.pandas()

In [110]:
prepared_data: pd.DataFrame = pd.read_csv('datasets/taghche.csv')
prepared_data = prepared_data[['comment', 'bookname']]

In [111]:
prepared_data.head()

,comment,bookname
0,اسم کتاب No one writes to the Colonel\nترجمش...,سرهنگ کسی ندارد برایش نامه بنویسد
1,"طاقچه عزیز،نام کتاب""کسی به سرهنگ نامه نمینویسد...",سرهنگ کسی ندارد برایش نامه بنویسد
2,بنظرم این اثر مارکز خیلی از صد سال تنهایی که ب...,سرهنگ کسی ندارد برایش نامه بنویسد
3,به نظر کتاب خوبی میومد اما من از ترجمش خوشم نی...,سرهنگ کسی ندارد برایش نامه بنویسد
4,کتاب خوبی است,سرهنگ کسی ندارد برایش نامه بنویسد


In [112]:
chars_stop_words = ''
with open('stopwords/chars.txt', 'r', encoding='utf-8') as file:
    chars_stop_words = ''.join(file.read().splitlines())

chars_stop_words = chars_stop_words.replace('[', '\[')
chars_stop_words = chars_stop_words.replace(']', '\]')
chars_pattern = re.compile(f'[{chars_stop_words}]')
chars_pattern

re.compile(r'[\ufeff!"#()*,-./:\[\]«»،؛؟۰۱۲۳۴۵۶۷۸۹…$ًٌٍَُِّْءٰٔ﷼]', re.UNICODE)

In [113]:
# https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
emojis_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+")
emojis_pattern

re.compile(r'[😀-🙏🌀-🗿🚀-\U0001f6ff\U0001f1e0-🇿]+', re.UNICODE)

In [114]:
def preprocess(text):
    global chars_pattern, emojis_pattern
    
    text = chars_pattern.sub(r' ', text)
    text = emojis_pattern.sub(r' ', text)
    return informal_normalizer_function(text)


# customizing InformalNormalizer().normalize()
# For seeing differences, you can see InformalNormalizer().normalize() method
def informal_normalizer_function(text):
    text = str(text)
    
    informal_normalizer = InformalNormalizer()
    text = Normalizer.normalize(informal_normalizer, text)
    sents = [
        informal_normalizer.word_tokenizer.tokenize(sentence)
        for sentence in informal_normalizer.sent_tokenizer.tokenize(text)
    ]

    normalized = [[informal_normalizer.normalized_word(word)[0] for word in sent] for sent in sents]
    normalized = np.array(normalized, dtype=object)
    return np.hstack(normalized)

In [115]:
prepared_data['comment'] = prepared_data['comment'].progress_apply(preprocess)

  0%|          | 0/69829 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
prepared_data.head()

In [96]:
ALL_PARTS_LEN = 18
crawled_data: pd.DataFrame = pd.read_csv('datasets/books data/books_data_part_1.csv')
for i in range(2, ALL_PARTS_LEN + 1):
    crawled_data = pd.concat([crawled_data, pd.read_csv(f'datasets/books data/books_data_part_{i}.csv')],
                             ignore_index=True)

In [97]:
# sort authors in order to drop duplicates
new_author_function = lambda x: ' $ '.join(sorted(str(x).split(' $ ')))
crawled_data['author'] = crawled_data['author'].apply(new_author_function)

In [98]:
before_dropping = len(crawled_data)
crawled_data = crawled_data.drop_duplicates()
print(f'Dropped {before_dropping - len(crawled_data)} duplicates.')

Dropped 2256 duplicates.


In [99]:
crawled_data = crawled_data.groupby(['name', 'author']).agg({'translator': set, 'publisher': set}).reset_index()

In [101]:
new_author_function = lambda x: set(x.split(' $ '))
crawled_data['author'] = crawled_data['author'].apply(new_author_function)

In [102]:
crawled_data.head()

,name,author,translator,publisher
0,\nسلوک معنوی ابن عربی,"{ویلیام سی. چیتیک, جان جی سولیوان}",{حسین مریدی},{انتشارات جامی}
1,100 پرسش و پاسخ درباره اختلال افسردگی یا دو ق...,{لیندا کاکرورتی},{nan},{نشر دانژه}
2,100 پرسش و پاسخ درباره نقص توجه/بیش فعالی بزر...,{آوا تی آلبرشت},{nan},{نشر دانژه}
3,12 شیوه اساسی برای ترکیب سواد رسانه و تفکر ان...,{سیدنی شیب},{مرجان اردشیرزاده},{سمندر}
4,25 اشتباه بچه گانه ای که زوج ها در زندگی زناش...,{پل دبلیو کلمن},{آرمانوش باباخانیانس},{دنیس}


In [103]:
data = pd.merge(prepared_data, crawled_data, left_on='bookname', right_on='name')
data = data.drop(columns=['bookname'])

In [104]:
print(f'Prepared data: {len(prepared_data)}\nCrawled data: {len(crawled_data)}\nMerged data: {len(data)}')
data.head()

Prepared data: 69829
Crawled data: 166540
Merged data: 88828


,comment,name,author,translator,publisher
0,اسم کتاب No one writes to the Colonel\nترجمش...,سرهنگ کسی ندارد برایش نامه بنویسد,{گابریل گارسیا مارکز},{نازنین نوذری},{موسسه فرهنگی هنری نوروز هنر}
1,"طاقچه عزیز،نام کتاب""کسی به سرهنگ نامه نمینویسد...",سرهنگ کسی ندارد برایش نامه بنویسد,{گابریل گارسیا مارکز},{نازنین نوذری},{موسسه فرهنگی هنری نوروز هنر}
2,بنظرم این اثر مارکز خیلی از صد سال تنهایی که ب...,سرهنگ کسی ندارد برایش نامه بنویسد,{گابریل گارسیا مارکز},{نازنین نوذری},{موسسه فرهنگی هنری نوروز هنر}
3,به نظر کتاب خوبی میومد اما من از ترجمش خوشم نی...,سرهنگ کسی ندارد برایش نامه بنویسد,{گابریل گارسیا مارکز},{نازنین نوذری},{موسسه فرهنگی هنری نوروز هنر}
4,کتاب خوبی است,سرهنگ کسی ندارد برایش نامه بنویسد,{گابریل گارسیا مارکز},{نازنین نوذری},{موسسه فرهنگی هنری نوروز هنر}


In [105]:
crawled_books = set(crawled_data['name'].values)
prepared_books = prepared_data['bookname'].values

unavailable_books = [book for book in prepared_books if book not in crawled_books]
print(f'Unavailable books: {len(unavailable_books)}')

Unavailable books: 8102
Is the number of unavailable books equal to the difference between prepared and merged data?
False


# Labeling